# Exploratory Data Analysis for H1B visas - Wage effect

This is an initial analysis performed on a data set obtained on [Kaggle](https://www.kaggle.com/nsharan/h-1b-visa) and containing over 3 million rows. More data can be obtained from the *Office of Foreign Labor Certification* which outputs yearly data sets. See this [link](https://www.dol.gov/agencies/eta/foreign-labor/performance) for more performance indices.

For this section, we load the data set with 4 columns: case status, full time position, prevailing wage and year. The case status is one of

- CERTIFIED-WITHDRAWN
- WITHDRAWN
- CERTIFIED
- DENIED
- REJECTED (only 2 rows)
- INVALIDATED (only 1 row)
- PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED (only 15 rows)
- Not available (nan)

The full time position indicates whether or not a position is full time. The year ranges from 2011 to 2016.

Our final goal is to estimate the probability of having an H-1B visa application rejected. In order to demonstrate viability, we show that there is sufficient data to obtain meaningful results. In this notebook, we focus on the relationship between wages and applications. In particular, we compute a chi-square statistic to show that the is a clear relationship between wages and the probability of seeing and H-1B application denied.

In [62]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')
import ipywidgets as widgets
from scipy.stats import chi2_contingency

In [2]:
df = pd.read_csv('data/h1b_kaggle.csv',
                 usecols = ['CASE_STATUS', 'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'YEAR'])

## Wage Distribution

We plot wage distribution for full time and non full time positions. The plots are presented for the totality of the data, as well as by year.

Notes: We remove rows with wages of of 0 as well as those whose wages are above $250 000 (we consider very high wages to be outliers.). Doing so, we omit an insignificant amount of data (about 0.1\%).

In [31]:
%matplotlib inline

# Total vs. full time vs. not full time
df_wage = df[(df['PREVAILING_WAGE'] <= 250000) & (df['PREVAILING_WAGE'] != 0) & (df['PREVAILING_WAGE'].notna())].copy()
df_full_time = df_wage[df_wage['FULL_TIME_POSITION'] == 'Y']
df_part_time = df_wage[df_wage['FULL_TIME_POSITION'] == 'N']

#Widget Setup
w_out = [widgets.Output()]
for i in range(1, 7):
    w_out.append(widgets.Output())

#----------------------------------------------------------------------
##Total
with w_out[0]:
    fig, ax = plt.subplots(3, figsize=(12, 12))
    fig.suptitle('Wage distribution')
    
    #Total
    ax[0].hist(df_wage['PREVAILING_WAGE'], bins = 100, edgecolor = 'white')
    
    #Full Time
    ax[1].hist(df_full_time['PREVAILING_WAGE'], bins = 100, edgecolor = 'white')
    ax[1].set_title('Full time')
    
    #Part Time
    ax[2].hist(df_part_time['PREVAILING_WAGE'], bins = 100, edgecolor = 'white')
    ax[2].set_title('Not full time')

    plt.show(fig)
    
#----------------------------------------------------------------------
##By year
for i in range(1, 7):
    with w_out[i]:
        year = 2010 + i
        df_wage_year = df_wage[df_wage['YEAR'] == year]
        df_full_time_year = df_full_time[df_full_time['YEAR'] == year]
        df_part_time_year = df_part_time[df_part_time['YEAR'] == year]
        
        fig, ax = plt.subplots(3, figsize=(12, 12))
        fig.suptitle('Wage distribution in ' + str(year))
    
        #Total
        ax[0].hist(df_wage_year['PREVAILING_WAGE'], bins = 100, edgecolor = 'white')
    
        #Full Time
        ax[1].hist(df_full_time_year['PREVAILING_WAGE'], bins = 100, edgecolor = 'white')
        ax[1].set_title('Full time')
    
        #Part Time
        ax[2].hist(df_part_time_year['PREVAILING_WAGE'], bins = 100, edgecolor = 'white')
        ax[2].set_title('Not full time')
        
        plt.show(fig)

tabs = widgets.Tab(children = w_out)
tabs.set_title(0, 'Total')
for i in range(1, 7):
    tabs.set_title(i, str(2010+i))
display(tabs)

### Observations

We observe that the separation between full time and not full time does not make sense for the year 2016. In fact, it seems that the data was simply divided between wages above and below \$70 000.

Moreover, we can see that most applications that are not full time appeared in the year 2016. Indeed, see the following table.

In [32]:
df_wage[df_wage['FULL_TIME_POSITION'] == 'N'].groupby('YEAR').size().to_frame().T

YEAR,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0
0,16579,15617,13557,14182,15093,351128


This is probably due to some error in the data collection. We therefore will not separate between full time and non full time positions in the year 2016.

## Ratio of denied applications by wage

We will separate the wage data into 5 groups corresponding each corresponring to 20\% of the data. More specifically, we cut the data by quantiles. We then compare the number of denied versus certified applications for each bin. We will plot our data and perform a chi squared test.

For this initial analysis, we do not distinguish between full time and non full time applications.

In [76]:
%matplotlib inline

#Widget Setup
w_out = [widgets.Output()]
for i in range(1, 7):
    w_out.append(widgets.Output())

#----------------------------------------------------------------------
##Total

#Grouping by wage group
df_wage['q5'], bins = pd.qcut(df_wage['PREVAILING_WAGE'], q = 5, labels = [1, 2, 3, 4, 5], retbins = True)

#Count by wage group - denied vs. certified
q5_denied = df_wage[df_wage['CASE_STATUS'] == 'DENIED'].groupby('q5').size()
q5_cert = df_wage[(df_wage['CASE_STATUS'] == 'CERTIFIED') | (df_wage['CASE_STATUS'] == 'CERTIFIED-WITHDRAWN')].groupby('q5').size()

#Get ratio of denied/certified by group
q5_ratio = q5_denied/q5_cert

##Labels - to siplay wage range
labels = []
for i in range(5):
    labels.append('(' + str(bins[i]) + ', ' + str(bins[i+1]) + ']')

#Plot
with w_out[0]:
    plt.subplots(figsize=(12, 8))
    plt.bar(q5_ratio.index, q5_ratio.values, align = 'center', alpha = 0.7,  linewidth=0)
    plt.xticks([1, 2, 3, 4, 5], labels)
    plt.show()
    
#----------------------------------------------------------------------
##By year
for i in range(1, 7):
    with w_out[i]:
        year = 2010 + i
        
        #Grouping by wage group
        df_year = df_wage[df_wage['YEAR'] == year].copy()
        df_year['q5'], bins = pd.qcut(df_year['PREVAILING_WAGE'], q = 5, labels = [1, 2, 3, 4, 5], retbins = True)

        #Count by wage group - denied vs. certified
        q5_denied = df_year[df_year['CASE_STATUS'] == 'DENIED'].groupby('q5').size()
        q5_cert = df_year[(df_year['CASE_STATUS'] == 'CERTIFIED') | (df_year['CASE_STATUS'] == 'CERTIFIED-WITHDRAWN')].groupby('q5').size()

        #Get ratio of denied/certified by group
        q5_ratio = q5_denied/q5_cert
        
        
        ##Labels - to siplay wage range
        labels = []
        for i in range(5):
            labels.append('(' + str(bins[i]) + ', ' + str(bins[i+1]) + ']')
    
        #Delete copy
        del df_year
        
        plt.subplots(figsize=(12, 8))
        plt.bar(q5_ratio.index, q5_ratio.values, align = 'center', alpha = 0.7,  linewidth=0)
        plt.xticks([1, 2, 3, 4, 5], labels)
        plt.show()

tabs = widgets.Tab(children = w_out)
tabs.set_title(0, 'Total')
for i in range(1, 7):
    tabs.set_title(i, str(2010+i))
display(tabs)

### Chi-Squared Test

We perform a chi-squared test to see if the number of denied and certified applications is independent of wage group.

In [72]:
%matplotlib inline

#Widget Setup
w_out = [widgets.Output()]
for i in range(1, 7):
    w_out.append(widgets.Output())

#----------------------------------------------------------------------
##Total

#Count by wage group - denied vs. certified
q5_denied = df_wage[df_wage['CASE_STATUS'] == 'DENIED'].groupby('q5').size()
q5_cert = df_wage[(df_wage['CASE_STATUS'] == 'CERTIFIED') | (df_wage['CASE_STATUS'] == 'CERTIFIED-WITHDRAWN')].groupby('q5').size()

#Perform Chi squared test
stat, p, dof, expected = chi2_contingency([q5_denied, q5_cert])

#Print
with w_out[0]:
    print('The chi square statistic is:', stat)
    print('This yields a p-value of:', p)
    
#----------------------------------------------------------------------
##By year
for i in range(1, 7):
    with w_out[i]:
        year = 2010 + i
        
        #data frame by year ad grouping
        df_year = df_wage[df_wage['YEAR'] == year].copy()
        df_year['q5'] = pd.qcut(df_year['PREVAILING_WAGE'], q = 5, labels = [1, 2, 3, 4, 5])
        
        #Count by wage group - denied vs. certified
        q5_denied = df_year[df_year['CASE_STATUS'] == 'DENIED'].groupby('q5').size()
        q5_cert = df_year[(df_year['CASE_STATUS'] == 'CERTIFIED') | (df_year['CASE_STATUS'] == 'CERTIFIED-WITHDRAWN')].groupby('q5').size()

        #Perform Chi squared test
        stat, p, dof, expected = chi2_contingency([q5_denied, q5_cert])
        
        #Delete copied data
        del df_year
        
        #Print
        print('The chi square statistic is:', stat)
        print('This yields a p-value of:', p)
        

tabs = widgets.Tab(children = w_out)
tabs.set_title(0, 'Total')
for i in range(1, 7):
    tabs.set_title(i, str(2010+i))
display(tabs)

#### Conclusion

We see that there is a correlation between wage groups and the ratio of denied:certified applications.